In [1]:
from text_processing import *
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
embeding_file="./glove.840B.300d.txt"
embed_size = 300
max_words = 20000
sentence_size = 30
model_file = "./best_model.hdf5"
is_first = 1

In [2]:
if is_first:
    neg = pd.read_csv("./data/MR/mr.neg", delimiter="\n", encoding="ISO-8859-1", header=None)
    pos = pd.read_csv("./data/MR/mr.pos", delimiter="\n", encoding="ISO-8859-1", header=None)

    neg = neg.rename(index=str, columns={0: "text"})
    pos = pos.rename(index=str, columns={0: "text"})

    neg["label"] = 0
    pos["label"] = 1

    data = pd.concat([neg,pos]).sample(frac=1).reset_index(drop=True)
    labels = data["label"].reshape(-1,1)
    word_sequence, word_index = tokenize(data["text"], max_words, sentence_size)
    train_data, test_data = word_sequence[:word_sequence.shape[0]-1000], word_sequence[-1000:]

    word_embeding,max_words = get_word_embding(word_index, embeding_file, max_words, embed_size)

    np.save("embeding-300d",word_embeding)
    np.save("train_data",train_data)
    np.save("test_data",test_data)
    np.save("labels",labels)
else:
    word_embeding = np.load("embeding-300d.npy")
    train_data = np.load("train_data.npy")
    test_data = np.load("test_data.npy")
    labels = np.load("labels.npy")
    max_words = word_embeding.shape[0]

/home/LAB/zuogs/mypyenv/lib/python3.5/site-packages/ipykernel_launcher.py:12: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if sys.path[0] == '':
/home/LAB/zuogs/mypyenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from algo import textrnn_algo
import tensorflow as tf
tf.reset_default_graph()
algo = textrnn_algo(train_data,labels[:train_data.shape[0]], word_embeding, model_file, classes=1, sentence_length=sentence_size,
                    embed_size=embed_size, target_is_prob=False, word_size=max_words,
                    is_sigmoid_loss=True, learning_rate=1e-3, epoch=20, early_stopping_rounds=5,optimizer=tf.train.AdamOptimizer,
                    batch_size=64)
algo.model_train()
result = algo.model_predict(test_data)

model has been created.


136it [00:11, 11.92it/s]


Epoch 1: train loss : 0.505422, val loss: 0.467052
*** New best score ***



136it [00:10, 12.37it/s]


Epoch 2: train loss : 0.403821, val loss: 0.410470
*** New best score ***



...

Epoch 18: train loss : 0.000263, val loss: 0.000763
*** New best score ***



136it [00:10, 12.44it/s]


Epoch 19: train loss : 0.000204, val loss: 0.000599
*** New best score ***



136it [00:10, 12.66it/s]


Epoch 20: train loss : 0.000158, val loss: 0.000472
*** New best score ***

INFO:tensorflow:Restoring parameters from ./best_model.hdf5
<class 'numpy.ndarray'>


In [4]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
print("roc_auc_score : ", roc_auc_score(labels[-1000:], result))
pred_label = result>0.5
print("accuracy : ",accuracy_score(labels[-1000:], pred_label))

roc_auc_score :  0.8964933761541549
accuracy :  0.816
